In [38]:
# Import dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
from pprint import pprint
import pymongo
import pandas as pd
import requests
import time
from webdriver_manager.chrome import ChromeDriverManager

In [9]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
There is no [win32] chromedriver for browser 93.0.4577 in cache
Get LATEST driver version for 93.0.4577
Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.63/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\david\.wdm\drivers\chromedriver\win32\93.0.4577.63]


In [11]:
# visit site and scrape
site_url = "https://mars.nasa.gov/news/"
browser.visit(url)

html = browser.html
soup = bs(html, 'html.parser')

<h2>NASA Mars News</h2>

In [12]:
#Scrape the Mars News Site and collect the latest News Title and Paragraph Text. 
#Assign the text to variables that you can reference later.

article = soup.find("div", class_="list_text")
summary = article.find("div", class_="article_teaser_body").text
title = article.find("div", class_="content_title").text
date = article.find("div", class_="list_date").text
print(date)
print(title)
print(summary)

September 23, 2021
NASA's Perseverance Rover Cameras Capture Mars Like Never Before
Scientists tap into an array of imagers aboard the six-wheeled explorer to get a big picture of the Red Planet.


<h2>JPL Mars Space Images - Featured Image</h2>

In [15]:
# Visit the url for the Featured Space Image site
image_url = "https://spaceimages-mars.com/"
browser.visit(image_url)

# Use splinter to navigate the site and find the image url for the current Featured Mars Image 
# Assign the url string to a variable called featured_image_url
html = browser.html
soup = bs(html, 'html.parser')
image = soup.find("img", class_="headerimage fade-in")["src"]
img_url = "https://spaceimages-mars.com/"+image
featured_image_url = img_url

In [16]:
featured_image_url 

'https://spaceimages-mars.com/image/featured/mars2.jpg'

<h2>Mars Facts</h2>

In [32]:
# Visit the Mars Facts webpage and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
df = pd.read_html("https://galaxyfacts-mars.com/")[1]
df.columns= ["Description", "Mars"]
df.set_index("Description", inplace=True)
df

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 ( Phobos & Deimos )
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [33]:
# Use Pandas to convert the data to a HTML table string.
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 ( Phobos &amp; Deimos )</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>

<h2>Mars Hemispheres</h2>

In [35]:
# Visit the astrogeology site to obtain high resolution images for each of Mar's hemispheres.
astro_url = "https://marshemispheres.com/"
browser.visit(astro_url)

In [40]:
#Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. 
#Use a Python dictionary to store the data using the keys img_url and title.


hemisphere_image_urls = []

for i in range (4):
    time.sleep(5)
    images = browser.find_by_tag('h3')
    images[i].click()
    html = browser.html
    soup = bs(html, 'html.parser')
    partial = soup.find("img", class_="wide-image")["src"]
    img_title = soup.find("h2",class_="title").text
    img_url = 'https://marshemispheres.com'+ partial
    dictionary={"title":img_title,"img_url":img_url}
    hemisphere_image_urls.append(dictionary)
    browser.back()

In [41]:
#Append the dictionary with the image url string and the hemisphere title to a list.
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.comimages/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.comimages/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.comimages/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.comimages/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]


In [42]:
browser.quit()